<a href="https://colab.research.google.com/github/KaavyaRekanar/Subjective_Scoring_Framework_VQA/blob/main/Subjective_Scoring_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ae649d402ba5dfc9fe598b36af43cf0e9ac817c1de10aa5890a0bdf7b28c1789
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util,models
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch import nn


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Read and store content
# of an excel file
read_file = pd.read_excel ("/content/drive/MyDrive/Subjective_scoring.xlsx")

# Write the dataframe object
# into csv file
read_file.to_csv ("/content/drive/MyDrive/VQA.csv",
                   index = None,
                header=True)

# read csv file and convert
# into a dataframe object
df = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/VQA.csv"))

# show the dataframe
df

,Questions,Answers,ViLT,ViLT_Score,LXMERT,LXMERT_Score,ViLBERT,ViLBERT_Score
0,why is everyone carrying an umbrella?,due to rain,raining,1.00,raining,1.00,shade,0.00
1,is there sand in the image?,yes,yes,1.00,yes,1.00,no,0.00
2,what is the number on the bus?,x50,55,0.25,1,0.00,0,0.00
3,how many animals are in the image?,5,5,1.00,4,0.75,0,0.00
4,what is the fruit in the image?,bananas,bananas,1.00,bananas,1.00,apple,0.25
5,what is the name of the boat?,lady joan III,fishing boat,0.00,marker,0.00,ship,0.00
6,who is on the skateboard?,man,man,1.00,man,1.00,nobody,0.00
7,what is the time on the wall clock?,11:30:00,09:05:00,0.00,01:50:00,0.00,0,0.00
8,why are people under umbrellas?,its sunny,shade,0.00,sunny,1.00,they aren't,0.00
9,how many people are talking on the phone?,1,0,0.00,2,0.00,0,0.00


##BERT based Models

In [ ]:
model1 =SentenceTransformer('nli-distilbert-base')

## Function for Embedding Generation and Cosine Similarity

In [ ]:
def pred(model,q_s, ans):
    try:
      return util.cos_sim(model.encode(q_s), model.encode(ans))[0].cpu().item()
    except:
      return 0.0

In [ ]:
df['predict_nli_ViLT'] = df.apply(lambda row: pred(model1, row['Answers'],row['ViLT']), axis=1)

df['predict_nli_LXMERT'] = df.apply(lambda row: pred(model1, row['Answers'],row['LXMERT']), axis=1)

df['predict_nli_ViLBERT'] = df.apply(lambda row: pred(model1, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Questions,Answers,ViLT,ViLT_Score,LXMERT,LXMERT_Score,ViLBERT,ViLBERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT
0,why is everyone carrying an umbrella?,due to rain,raining,1.00,raining,1.00,shade,0.00,0.874358,0.874358,0.486050
1,is there sand in the image?,yes,yes,1.00,yes,1.00,no,0.00,1.000000,1.000000,0.763509
2,what is the number on the bus?,x50,55,0.25,1,0.00,0,0.00,0.795372,0.640652,0.708958
3,how many animals are in the image?,5,5,1.00,4,0.75,0,0.00,1.000000,0.757595,0.753155
4,what is the fruit in the image?,bananas,bananas,1.00,bananas,1.00,apple,0.25,1.000000,1.000000,0.739052
5,what is the name of the boat?,lady joan III,fishing boat,0.00,marker,0.00,ship,0.00,0.204745,0.451815,0.454125
6,who is on the skateboard?,man,man,1.00,man,1.00,nobody,0.00,1.000000,1.000000,0.484185
7,what is the time on the wall clock?,11:30:00,09:05:00,0.00,01:50:00,0.00,0,0.00,0.799867,0.811588,0.453588
8,why are people under umbrellas?,its sunny,shade,0.00,sunny,1.00,they aren't,0.00,0.742271,0.986932,0.412427
9,how many people are talking on the phone?,1,0,0.00,2,0.00,0,0.00,0.853675,0.885801,0.853675


In [ ]:
def pearson_correlation(x,y):
    mean_x = sum(x) / len(x)
    mean_y = sum(y) / len(y)
    cov = sum((a - mean_x) * (b - mean_y) for (a, b) in zip(x, y)) / len(x)

    std_x, std_y = np.std(x), np.std(y)

    p = cov / (std_x * std_y)

    return float(p)

In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['predict_nli_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['predict_nli_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['predict_nli_ViLBERT']))

Pearson Correlation for ViLT: 0.6685710209290805
Pearson Correlation for LXMERT: 0.722930971874474
Pearson Correlation for ViLBERT: 0.27084137327172136


In [ ]:
model2= SentenceTransformer('all-mpnet-base-v2') #This one is identified as one of the best performing model in SBERT documentation

In [ ]:
df['predict_mpnet_ViLT'] = df.apply(lambda row: pred(model2, row['Answers'],row['ViLT']), axis=1)

df['predict_mpnet_LXMERT'] = df.apply(lambda row: pred(model2, row['Answers'],row['LXMERT']), axis=1)

df['predict_mpnet_ViLBERT'] = df.apply(lambda row: pred(model2, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Questions,Answers,ViLT,ViLT_Score,LXMERT,LXMERT_Score,ViLBERT,ViLBERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT
0,why is everyone carrying an umbrella?,due to rain,raining,1.00,raining,1.00,shade,0.00,0.874358,0.874358,0.486050,0.705674,0.705674,0.231433
1,is there sand in the image?,yes,yes,1.00,yes,1.00,no,0.00,1.000000,1.000000,0.763509,1.000000,1.000000,0.700037
2,what is the number on the bus?,x50,55,0.25,1,0.00,0,0.00,0.795372,0.640652,0.708958,0.429984,0.165352,0.205616
3,how many animals are in the image?,5,5,1.00,4,0.75,0,0.00,1.000000,0.757595,0.753155,1.000000,0.796634,0.501212
4,what is the fruit in the image?,bananas,bananas,1.00,bananas,1.00,apple,0.25,1.000000,1.000000,0.739052,1.000000,1.000000,0.392612
5,what is the name of the boat?,lady joan III,fishing boat,0.00,marker,0.00,ship,0.00,0.204745,0.451815,0.454125,0.171492,0.114703,0.229577
6,who is on the skateboard?,man,man,1.00,man,1.00,nobody,0.00,1.000000,1.000000,0.484185,1.000000,1.000000,0.183367
7,what is the time on the wall clock?,11:30:00,09:05:00,0.00,01:50:00,0.00,0,0.00,0.799867,0.811588,0.453588,0.764570,0.611121,0.225404
8,why are people under umbrellas?,its sunny,shade,0.00,sunny,1.00,they aren't,0.00,0.742271,0.986932,0.412427,0.463302,0.734968,0.091107
9,how many people are talking on the phone?,1,0,0.00,2,0.00,0,0.00,0.853675,0.885801,0.853675,0.613616,0.775843,0.613616


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['predict_mpnet_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['predict_mpnet_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['predict_mpnet_ViLBERT']))

Pearson Correlation for ViLT: 0.7915879831254405
Pearson Correlation for LXMERT: 0.740680911882671
Pearson Correlation for ViLBERT: 0.09506198873243031


##Basic BERT Model

In [ ]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())

model3 = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

In [ ]:
df['bert_ViLT'] = df.apply(lambda row: pred(model3, row['Answers'],row['ViLT']), axis=1)

df['bert_LXMERT'] = df.apply(lambda row: pred(model3, row['Answers'],row['LXMERT']), axis=1)

df['bert_ViLBERT'] = df.apply(lambda row: pred(model3, row['Answers'],row['ViLBERT']), axis=1)

In [ ]:
df

,Questions,Answers,ViLT,ViLT_Score,LXMERT,LXMERT_Score,ViLBERT,ViLBERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT,bert_ViLT,bert_LXMERT,bert_ViLBERT
0,why is everyone carrying an umbrella?,due to rain,raining,1.00,raining,1.00,shade,0.00,0.874358,0.874358,0.486050,0.705674,0.705674,0.231433,0.383221,0.383221,0.394992
1,is there sand in the image?,yes,yes,1.00,yes,1.00,no,0.00,1.000000,1.000000,0.763509,1.000000,1.000000,0.700037,1.000000,1.000000,0.865070
2,what is the number on the bus?,x50,55,0.25,1,0.00,0,0.00,0.795372,0.640652,0.708958,0.429984,0.165352,0.205616,0.620574,0.637502,0.712907
3,how many animals are in the image?,5,5,1.00,4,0.75,0,0.00,1.000000,0.757595,0.753155,1.000000,0.796634,0.501212,1.000000,0.930467,0.764546
4,what is the fruit in the image?,bananas,bananas,1.00,bananas,1.00,apple,0.25,1.000000,1.000000,0.739052,1.000000,1.000000,0.392612,1.000000,1.000000,0.899500
5,what is the name of the boat?,lady joan III,fishing boat,0.00,marker,0.00,ship,0.00,0.204745,0.451815,0.454125,0.171492,0.114703,0.229577,0.484954,0.440914,0.450035
6,who is on the skateboard?,man,man,1.00,man,1.00,nobody,0.00,1.000000,1.000000,0.484185,1.000000,1.000000,0.183367,1.000000,1.000000,0.864310
7,what is the time on the wall clock?,11:30:00,09:05:00,0.00,01:50:00,0.00,0,0.00,0.799867,0.811588,0.453588,0.764570,0.611121,0.225404,0.929643,0.907619,0.328377
8,why are people under umbrellas?,its sunny,shade,0.00,sunny,1.00,they aren't,0.00,0.742271,0.986932,0.412427,0.463302,0.734968,0.091107,0.786809,0.808748,0.592084
9,how many people are talking on the phone?,1,0,0.00,2,0.00,0,0.00,0.853675,0.885801,0.853675,0.613616,0.775843,0.613616,0.846308,0.933066,0.846308


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['bert_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['bert_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['bert_ViLBERT']))

Pearson Correlation for ViLT: 0.29466458826523
Pearson Correlation for LXMERT: 0.25157217763237116
Pearson Correlation for ViLBERT: 0.3716174088014881


##GPT2 Model -- GPT3 is not freely available

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)


# Generate embeddings for the text pairs in the DataFrame
embeddings_list = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['ViLT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_ViLT'] = cosine_similarities

# The DataFrame now contains text embeddings and cosine similarity values




In [ ]:
embeddings_list1 = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['LXMERT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list1.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list1)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities1 = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities1.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_LXMERT'] = cosine_similarities1

# The DataFrame now contains text embeddings and cosine similarity values


In [ ]:
embeddings_list2 = []

for _, row in df.iterrows():
    text1 = row['Answers']
    text2 = row['ViLBERT']

    # Tokenize and encode the texts using the GPT-2 tokenizer
    inputs1 = tokenizer(text1, return_tensors='pt', padding=False, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=False, truncation=True)

    # Generate embeddings for the texts using GPT-2
    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    embeddings_list2.append((embeddings1, embeddings2))

# Add embeddings to the DataFrame
df['embeddings1'], df['embeddings2'] = zip(*embeddings_list2)

# Calculate cosine similarity between the embeddings and store it in a new column
cosine_similarities2 = []

for _, row in df.iterrows():
    similarity = cosine_similarity(row['embeddings1'].numpy(), row['embeddings2'].numpy())
    cosine_similarities2.append(similarity[0][0])

df.drop(columns=['embeddings1', 'embeddings2'], inplace=True)

df['gpt2_ViLBERT'] = cosine_similarities2


In [ ]:
df


,Questions,Answers,ViLT,ViLT_Score,LXMERT,LXMERT_Score,ViLBERT,ViLBERT_Score,predict_nli_ViLT,predict_nli_LXMERT,predict_nli_ViLBERT,predict_mpnet_ViLT,predict_mpnet_LXMERT,predict_mpnet_ViLBERT,bert_ViLT,bert_LXMERT,bert_ViLBERT,gpt2_ViLT,gpt2_LXMERT,gpt2_ViLBERT
0,why is everyone carrying an umbrella?,due to rain,raining,1.00,raining,1.00,shade,0.00,0.874358,0.874358,0.486050,0.705674,0.705674,0.231433,0.383221,0.383221,0.394992,0.955123,0.955123,0.990850
1,is there sand in the image?,yes,yes,1.00,yes,1.00,no,0.00,1.000000,1.000000,0.763509,1.000000,1.000000,0.700037,1.000000,1.000000,0.865070,1.000000,1.000000,0.993065
2,what is the number on the bus?,x50,55,0.25,1,0.00,0,0.00,0.795372,0.640652,0.708958,0.429984,0.165352,0.205616,0.620574,0.637502,0.712907,0.987077,0.991666,0.988131
3,how many animals are in the image?,5,5,1.00,4,0.75,0,0.00,1.000000,0.757595,0.753155,1.000000,0.796634,0.501212,1.000000,0.930467,0.764546,1.000000,0.999751,0.998759
4,what is the fruit in the image?,bananas,bananas,1.00,bananas,1.00,apple,0.25,1.000000,1.000000,0.739052,1.000000,1.000000,0.392612,1.000000,1.000000,0.899500,1.000000,1.000000,0.983611
5,what is the name of the boat?,lady joan III,fishing boat,0.00,marker,0.00,ship,0.00,0.204745,0.451815,0.454125,0.171492,0.114703,0.229577,0.484954,0.440914,0.450035,0.992653,0.996993,0.982080
6,who is on the skateboard?,man,man,1.00,man,1.00,nobody,0.00,1.000000,1.000000,0.484185,1.000000,1.000000,0.183367,1.000000,1.000000,0.864310,1.000000,1.000000,0.986161
7,what is the time on the wall clock?,11:30:00,09:05:00,0.00,01:50:00,0.00,0,0.00,0.799867,0.811588,0.453588,0.764570,0.611121,0.225404,0.929643,0.907619,0.328377,0.999799,0.999872,0.980235
8,why are people under umbrellas?,its sunny,shade,0.00,sunny,1.00,they aren't,0.00,0.742271,0.986932,0.412427,0.463302,0.734968,0.091107,0.786809,0.808748,0.592084,0.992723,0.990603,0.944045
9,how many people are talking on the phone?,1,0,0.00,2,0.00,0,0.00,0.853675,0.885801,0.853675,0.613616,0.775843,0.613616,0.846308,0.933066,0.846308,0.999033,0.999731,0.999033


In [ ]:
print("Pearson Correlation for ViLT:", pearson_correlation(df['ViLT_Score'], df['gpt2_ViLT']))
print("Pearson Correlation for LXMERT:", pearson_correlation(df['LXMERT_Score'], df['gpt2_LXMERT']))
print("Pearson Correlation for ViLBERT:", pearson_correlation(df['ViLBERT_Score'], df['gpt2_ViLBERT']))

Pearson Correlation for ViLT: -0.14920516119193264
Pearson Correlation for LXMERT: -0.26096646257764183
Pearson Correlation for ViLBERT: -0.022131374979498752


## Mean Absolute Error

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print(" For NLI Model ")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['predict_nli_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['predict_nli_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['predict_nli_ViLBERT']))

 For NLI Model 
MAE for ViLT: 0.3271573007106781
MAE for LXMERT: 0.38948363065719604
MAE for ViLBERT: 0.5858723491430282


In [ ]:
print(" For mpnet Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['predict_mpnet_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['predict_mpnet_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['predict_mpnet_ViLBERT']))

 For mpnet Model
MAE for ViLT: 0.24872891306877137
MAE for LXMERT: 0.2818974979221821
MAE for ViLBERT: 0.31239821910858157


In [ ]:
print(" For BERT Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['bert_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['bert_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['bert_ViLBERT']))

 For BERT Model
MAE for ViLT: 0.40350680351257323
MAE for LXMERT: 0.41641613841056824
MAE for ViLBERT: 0.6468130439519882


In [ ]:
print(" For GPT-2 Model")
print("MAE for ViLT:", mean_absolute_error(df['ViLT_Score'],df['gpt2_ViLT']))
print("MAE for LXMERT:", mean_absolute_error(df['ViLT_Score'],df['gpt2_LXMERT']))
print("MAE for ViLBERT:", mean_absolute_error(df['ViLBERT_Score'],df['gpt2_ViLBERT']))

 For GPT-2 Model
MAE for ViLT: 0.47661612629890443
MAE for LXMERT: 0.4773990631103516
MAE for ViLBERT: 0.9595968902111054


## Root Mean Squared Error for All Models

In [ ]:
from sklearn.metrics import mean_squared_error
import math
#nli RMSE   "Sqaured= True (MSE) , squared= False (RMSE)"
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['predict_nli_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['predict_nli_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['predict_nli_ViLBERT'], squared =False)))

RMSE ViLT 0.6906034155036381
RMSE LXMERT 0.6748343750982743
RMSE ViLBERT 0.7785198686261005


In [ ]:
# mpnet RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['predict_mpnet_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['predict_mpnet_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['predict_mpnet_ViLBERT'], squared =False)))

RMSE ViLT 0.6032343414514737
RMSE LXMERT 0.5854695404715569
RMSE ViLBERT 0.6094287233595349


In [ ]:
##BERT RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['bert_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['bert_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['bert_ViLBERT'], squared =False)))

RMSE ViLT 0.73713357114989
RMSE LXMERT 0.7238125900657859
RMSE ViLBERT 0.8209939834296619


In [ ]:
#GPT-2 RMSE
print("RMSE ViLT",math.sqrt(mean_squared_error(df['ViLT_Score'],df['gpt2_ViLT'], squared =False)))
print("RMSE LXMERT",math.sqrt(mean_squared_error(df['LXMERT_Score'],df['gpt2_LXMERT'], squared =False)))
print("RMSE ViLBERT",math.sqrt(mean_squared_error(df['ViLBERT_Score'],df['gpt2_ViLBERT'], squared =False)))

RMSE ViLT 0.8196643164768883
RMSE LXMERT 0.7973047204370003
RMSE ViLBERT 0.9811541757437652
